In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionImg2ImgPipeline, AutoencoderKL,StableDiffusionPipeline

from transformers import AutoTokenizer, DistilBertForSequenceClassification
from diffusers import EulerDiscreteScheduler
import torch

In [ ]:
import torch
from PIL import Image
import numpy as np

In [ ]:
from embedding.embedding import EmbeddingExtent
from embedding.text_encoder_hijack import TextEncoderHijack
from embedding.textual_inversion import TextualInversionPlug

from cn_process import ControlNetProcess


In [ ]:
generator = DiffusionPipeline.from_pretrained("../model/diffuser_model/",torch_dtype=torch.float16,safety_checker=None)

# vae = AutoencoderKL.from_pretrained("vae_path",torch_dtype=torch.float16,subfolder="vae").to("cuda")
# generator.vae = vae

generator.scheduler = EulerDiscreteScheduler.from_config(generator.scheduler.config)

generator.to("cuda")

In [ ]:
CLIP_Tokenizer = generator.tokenizer
CLIP_TextModel = generator.text_encoder

In [ ]:
# load TextualInversion
TextualInversion = TextualInversionPlug('../model/textual_inversion',tokenizer = CLIP_Tokenizer)
TextualInversion.load_textual_inversion()

In [ ]:
hijack = TextEncoderHijack()
hijack.hijack_embeding(CLIP_TextModel)

In [ ]:
embedding = EmbeddingExtent(tokenizer=CLIP_Tokenizer,text_encoder=CLIP_TextModel,textual_inversion_manager=TextualInversion,hijack=hijack,device="cuda",dtype=torch.float16)


In [ ]:
prompt = "Masterpiece, high quality, high detail, (((realistic))),(3d:1.2), solo, detailed beautiful face and eyes, Chinese clothes, detailed background, water, illustrations, vivid colors, movie-like lighting"

negative = "Dark skin,((nsfw:1.3)),(EasyNegative:1.3),(badhandv4:1.5),(ng_deepnegative_v1_75t:1.2),(worst quality:1.5),(low quality:1.2),watermark,username,text,(cameltoe:1.3),((realistic:1.3)),((long pointy ears:1.3)),((forehead:1.3)),((watermark:1.3)),(((animal ears:1.3))),jacket"



In [ ]:
# CLIP_stop_at_last_layers is clip_skip in webui
prompt_pre_embedding = embedding(prompt,CLIP_stop_at_last_layers=1)
negative_prompt_pre_embedding = embedding(negative,CLIP_stop_at_last_layers=1)

In [ ]:
prompt_pre_embedding,negative_prompt_pre_embedding = embedding.pad_prompt_tensor_same_length(prompt_emb=prompt_pre_embedding, negative_prompt_emb=negative_prompt_pre_embedding,CLIP_stop_at_last_layers=1)




In [ ]:
preprocess_names=['canny']
preprocess_model_path='../model/controlNet/preprocess_model'
preprocess_params=None

In [ ]:
controlnet_names=['canny']
controlnet_model_path='../model/controlNet'

In [ ]:
cn = ControlNetProcess(generator=generator, controlnet_names=controlnet_names,preprocess_names=preprocess_names,controlnet_model_path=controlnet_model_path,preprocess_model_path=preprocess_model_path,device='cuda')




In [ ]:
apply_lora_list = []
cn_img = Image.open('../canny.png')
tile_img = Image.open('../tile.png')

In [ ]:
cn_info = {
    "cn_image":cn_img,
    "cn_model_name":"depth",
    "cn_model_scale":0.5,
    "cn_pre_model_name":"depth_leres++"
}

# cn_info = [{
#     "cn_image":cn_img,
#     "cn_model_name":"depth",
#     "cn_model_scale":0.5,
#     "cn_pre_model_name":"depth_leres++"
# },{
#     "cn_image":cn_img,
#     "cn_model_name":"canny",
#     "cn_model_scale":0.8,
#     "cn_pre_model_name":"canny"
# }]

In [ ]:
image,info = cn(prompt_embeds=prompt_pre_embedding,
                width=512,
                height=768,
                negative_prompt_embeds=negative_prompt_pre_embedding,
                prompt=prompt,
                negative_prompt=negative,
                apply_lora_list=apply_lora_list,
                controlnet_info=cn_info,
                guidance_scale=7,
                num_inference_steps=20,
                num_images_per_prompt=1,
               guess_mode=True)

In [ ]:
image[0]